In [1]:
# ignore warnings
import warnings
warnings.filterwarnings("ignore")

# Wrangling
import pandas as pd
import numpy as np
import os
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split

# Exploring
import scipy.stats as stats
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures

# Modeling
from sklearn.linear_model import LinearRegression, LassoLars, TweedieRegressor


# Visualizing
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

# default pandas decimal number display format
pd.options.display.float_format = '{:20,.2f}'.format

import math 
import wrangle_functions
import env
import acquire
import prepare

## Acquire

In [2]:
# using a SQL query I pull in the zillow data set for 2017 properties and predictions data for single 
# unit / single family homes from the Codeup database.
df = acquire.new_zillow_data()

In [3]:
# taking a look at the dataset
df.head()

,typeconstructiontypeid,storytypeid,propertylandusetypeid,heatingorsystemtypeid,buildingclasstypeid,architecturalstyletypeid,airconditioningtypeid,id,parcelid,basementsqft,...,id,parcelid,logerror,transactiondate,id,last_trans_date,propertylandusedesc,storydesc,typeconstructiondesc,parcelid
0,NaN,NaN,269.00,NaN,NaN,NaN,NaN,0,10754147,NaN,...,0,14297519,0.03,2017-01-01,0,2017-01-01,Planned Unit Development,None,None,10754147
1,NaN,NaN,261.00,NaN,NaN,NaN,NaN,1,10759547,NaN,...,1,17052889,0.06,2017-01-01,1,2017-01-01,Single Family Residential,None,None,10759547
2,NaN,NaN,47.00,NaN,5.00,NaN,NaN,2,10843547,NaN,...,2,14186244,0.01,2017-01-01,2,2017-01-01,Store/Office (Mixed Use),None,None,10843547
3,NaN,NaN,47.00,NaN,3.00,NaN,NaN,3,10859147,NaN,...,3,12177905,-0.10,2017-01-01,3,2017-01-01,Store/Office (Mixed Use),None,None,10859147
4,NaN,NaN,31.00,NaN,4.00,NaN,NaN,4,10879947,NaN,...,4,10887214,0.01,2017-01-01,4,2017-01-01,Commercial/Office/Residential Mixed Used,None,None,10879947


In [4]:
# Intially I will pull in 77,614 rows and 73 columns
df.shape

(77614, 73)

## Prepare

In [5]:
# here I have applied all of my preparation techniques from the prepare.py file using the prep_all_function
df, df_outs = prepare.prep_all_function(df)

With my prep_all_function started by summarizing the dataset, next I chose the necessary columns from the original 73 that best represented the information I thought might be useful to explore, cluster with and model on.  I imputed columns as I saw fit and dropped columns with more than 25% of missing values.  I removed outliers using a function that took the most extreme outliers from the main dataframe, but kept most other outliers and saved this df to use in modeling.

In [6]:
# after prepartion is performed my dataset now consists of 51,219 rows and 14 columns
df.shape

(57795, 14)

In [7]:
# here is my dataset that includes outliers that I might utilize in modeling
#df_outs.shape

In [8]:
train, validate, test, X_train, y_train, X_validate, y_validate, X_test, y_test = prepare.train_validate_test(df, "logerror")

In [9]:
train.shape

(32365, 14)

In [10]:
Conclusions and Next Steps
Results:
Best predictors discovered were bedroom count, number of bedrooms, number of bathrooms, and square footage
Best-peforming model outperformed baseline and increased $R^2$ value by 0.2%
Next Steps:
Try more feature combinations for clustering
More than 3 features
Try more feature combinations in models

SyntaxError: invalid syntax (<ipython-input-10-c6b80794ae0f>, line 1)